In [1]:
from alpaca_trade_api.rest import REST
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from datetime import datetime, timedelta
from pytz import timezone

from feature_extraction import *
from feature_selection import *
from backtest import *
from labelling import *
import importlib
from models import *
from visualisation import *
import yfinance

ModuleNotFoundError: No module named 'feature_extraction'

In [ ]:
# the point of this notebook is to showcase my skillset
# I dont have to show my shitty 1 minute bar strategies
# a day strategy that beats the S&P 500 is good pitch
# show steps of analysis and visualisation
# run RFE to find optimal parameters
# show how the strategy is profitable

In [2]:
import yfinance

In [2]:
API_KEY = "PKCX5E3WZD8B92RHZ145"
SECRET_KEY = "KGmHFAJdwbwGaqbBofxwQcuW6Ew4q4qbbJ7XcUIa"
api = REST(key_id=API_KEY,secret_key=SECRET_KEY,base_url="https://paper-api.alpaca.markets")

In [3]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [4]:
symbol = ['PLTR']
hist=api.get_bars(symbol,timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Minute), start='2022-05-01').df
hist = hist.drop(columns = ['symbol'])

In [ ]:
symbol = ['F']
hist=api.get_bars(symbol,timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Minute), start='2022-05-01').df
hist = hist.drop(columns = ['symbol'])

In [ ]:
symbol = ['LUMN']
hist=api.get_bars(symbol,timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Minute), start='2022-05-01').df
hist = hist.drop(columns = ['symbol'])

In [ ]:
symbol = ['BTCUSD']
hist=api.get_crypto_bars(symbol,timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Minute),start='2022-05-01', exchanges=['CBSE']).df
hist = hist.drop(columns = ['exchange','symbol'])

In [5]:
hist.index = hist.index - timedelta(hours = 4)
#hist.index = hist.index.map(lambda t: t.replace(tzinfo=timezone('America/New_York')))

In [6]:
hist = hist.between_time('9:30', '16:00')

In [7]:
importlib.reload(feature_extraction)

<module 'feature_extraction' from '/Users/gluzman/Desktop/Code/stonks/stonks/feature_extraction.py'>

In [ ]:
hist

In [8]:
# compute various indicators
import feature_extraction
#hist = feature_extraction.get_features(hist)
hist = feature_extraction.get_features(hist, window_limit=9000)

[1, 2, 3, 5, 8, 12, 20, 26, 40, 50, 60, 80, 100, 150, 200, 250, 300, 400, 500, 600, 700, 800, 1000, 1200, 1500, 1800, 2000, 2500, 3000, 5000, 6000, 7000, 9000, 10000, 12000, 15000, 20000, 30000, 45000, 60000, 80000, 100000, 150000, 200000]
[[1, 2], [2, 3], [3, 5], [5, 8], [5, 12], [5, 26], [8, 26], [12, 26], [12, 50], [12, 100], [12, 500], [20, 60], [20, 150], [26, 50], [26, 80], [26, 100], [26, 500], [50, 100], [50, 200], [60, 150], [60, 1500], [300, 500], [300, 800], [700, 1500], [1500, 15000], [7000, 20000], [10000, 45000], [45000, 100000], [45000, 200000]]


/usr/local/lib/python3.10/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/usr/local/lib/python3.10/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
/usr/local/lib/python3.10/site-packages/ta/trend.py:787: RuntimeWarning: invalid value encountered in divide
  directional_index = 100 * np.abs((dip - din) / (dip + din))
/usr/local/lib/python3.10/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/usr/local/lib/python3.10/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)
/usr/local/lib/python3.10/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/usr/local/lib/python3.1

In [ ]:
#hist = hist.dropna()

In [ ]:
from stonks import labelling
# label data for potential ML manipulations
# lets use volatility to label
# hist['higher_bound'] = hist.close + hist.vol_std_12 * 1
# hist['lower_bound'] = hist.close - hist.vol_std_12 * 1

# hist['higher_bound'] = hist.close + hist.vol_std_12.ewm(50).mean() * 1
# hist['lower_bound'] = hist.close - hist.vol_std_12.ewm(50).mean() * 1


# hist['higher_bound'] = hist.close + hist.vol_std_50 * 1
# hist['lower_bound'] = hist.close - hist.vol_std_50 * 1

# hist['higher_bound'] = hist.close + hist.vol_std_50 * 3
# hist['lower_bound'] = hist.close - hist.vol_std_50 * 1

# hist['higher_bound'] = hist.close + (hist.close - hist.swing_low_50) * 1.5
# hist['lower_bound'] = hist.close - (hist.close - hist.swing_low_50) * 1

# hist['higher_bound'] = hist.close + (hist.close - hist.swing_low_200) * 1.5
# hist['lower_bound'] = hist.close - (hist.close - hist.swing_low_200) * 1

#hist = get_barrier(hist,hist.close.values,hist.higher_bound.values,hist.lower_bound.values,window=30)
#hist = get_barrier(hist,hist.close.values,hist.higher_bound.values,hist.lower_bound.values,window=100)
hist = labelling.get_barrier(hist,hist.close.values,hist.higher_bound.values,hist.lower_bound.values,window=12,side=None)

#hist = get_barrier(hist,hist.close.values,hist.higher_bound.values,hist.lower_bound.values,window=150)
#hist = get_barrier(hist,hist.close.values,hist.higher_bound.values,hist.lower_bound.values,window=50)

# try new targets - 
# try regression vs classification
# try forecast positive 1 min increase
# try forecast positive 5 min increase
# use pct change - fixed or volatility based
# forecast future ema - current ema > pct_thresh

In [ ]:
# # this version did not work as well
# # flat future price

# window = 12

# multiplier = np.abs((1 - (hist.close/ hist.close.shift(window)).ewm(window * 10).mean()))
# # try flat multiplier here
# # try flat % gain on closing price
# hist['higher_bound'] = hist.close * (1 + (multiplier * 1))
# hist['lower_bound'] = hist.close * (1 - (multiplier * 1))

# future_price = hist.close.shift(-window)
# hist.barrier = 0
# hist.barrier.loc[hist['higher_bound'] < future_price] = 1
# hist.barrier.loc[hist['lower_bound'] > future_price] = -1

In [ ]:
#
#np.abs((1 - (hist['ema_' + str(window)]/ hist['ema_' + str(window)].shift(window)).ewm(window * 100).mean())).describe()


# current version
# takes the average 12 min pct shift over the last 1200 mins, BUT averages gains and losses
# so if we lose typically 10% it will be 1.1
# if we win it will be 1.1
# but if it is 50/50 it will be 1
# this means that the algorithm will train to predict big swings when the market is trending up/ down and small swings the more stable it is

# what if
# exp_ema_pct_abs_shift_'
#(np.abs(1 - hist['ema_' + str(window)]/ hist['ema_' + str(window)].shift(window))).ewm(window * 100).mean().describe()

# exp_ema_pct_dir_shift_'
#((1 - hist['ema_' + str(window)]/ hist['ema_' + str(window)].shift(window))).ewm(window * 100).mean().describe()

In [ ]:
window = 50

# 100 or 10??
# hist['multiplier'] = np.abs((1 - (hist['ema_' + str(window)]/ hist['ema_' + str(window)].shift(window)).ewm(window * 100).mean()))
# hist['multiplier_volatility'] = hist['vol_std_' + str(window)]/hist['ema_' + str(window)].ewm(window * 10).mean()
#hist['multiplier'] = hist['exp_ema_pct_shift_12']

hist['multiplier'] = hist['exp_ema_pct_abs_shift_50']
hist['multiplier_volatility'] = hist['exp_pct_volatility_50']

# try flat multiplier here
# try flat % gain on closing price
hist['higher_bound'] = hist.close * (1 + (hist.multiplier * 0.3))
hist['lower_bound'] = hist.close * (1 - (hist.multiplier * 0.3))

future_ema = hist['ema_' + str(window)].shift(-window)
hist['barrier'] = 0
hist['barrier'].loc[hist['higher_bound'] < future_ema] = 1
hist['barrier'].loc[hist['lower_bound'] > future_ema] = -1

In [ ]:
window = 26
# add min max here
hist['multiplier'] = hist['exp_ema_pct_abs_shift_12']
hist['multiplier_volatility'] = hist['exp_pct_volatility_26']

# try flat multiplier here
# try flat % gain on closing price
hist['higher_bound'] = hist.close * (1 + (hist.multiplier_volatility * 2))
hist['lower_bound'] = hist.close * (1 - (hist.multiplier_volatility * 1))

future_min = hist['swing_low_' + str(window)].shift(-window)
future_max = hist['swing_high_' + str(window)].shift(-window)
hist['barrier'] = 0
hist['barrier'].loc[hist['higher_bound'] < future_max] = 1
#hist['barrier'].loc[hist['lower_bound'] > future_min] = -1

In [ ]:
# absolute price
from stonks import labelling
window = 12

# 100 or 10??
# hist['multiplier'] = np.abs((1 - (hist['ema_' + str(window)]/ hist['ema_' + str(window)].shift(window)).ewm(window * 100).mean()))
# hist['multiplier_volatility'] = hist['vol_std_' + str(window)]/hist['ema_' + str(window)].ewm(window * 10).mean()
#hist['multiplier'] = hist['exp_ema_pct_shift_12']

hist['multiplier'] = hist['exp_ema_pct_abs_shift_12']
hist['multiplier_volatility'] = hist['exp_pct_volatility_12']
hist['multiplier_volatility'] = hist['exp_pct_volatility_12']

# try flat multiplier here
# try flat % gain on closing price
hist['higher_bound'] = hist.close * (1 + (hist.multiplier_volatility * 1))
hist['lower_bound'] = hist.close * (1 - (hist.multiplier_volatility * 1))

hist = labelling.get_barrier(hist,hist.close.values,hist.higher_bound.values,hist.lower_bound.values,window=12,side=None)

In [9]:
# absolute price
import labelling
window = 50

# 100 or 10??
# hist['multiplier'] = np.abs((1 - (hist['ema_' + str(window)]/ hist['ema_' + str(window)].shift(window)).ewm(window * 100).mean()))
# hist['multiplier_volatility'] = hist['vol_std_' + str(window)]/hist['ema_' + str(window)].ewm(window * 10).mean()
#hist['multiplier'] = hist['exp_ema_pct_shift_12']

hist['multiplier'] = hist['exp_ema_pct_abs_shift_50']
hist['multiplier_volatility'] = hist['exp_pct_volatility_50']
hist['multiplier_volatility'] = hist['exp_pct_volatility_50']

# try flat multiplier here
# try flat % gain on closing price
# hist['higher_bound'] = hist.close * (1 + (hist.multiplier_volatility * 1))
# hist['lower_bound'] = hist.close * (1 - (hist.multiplier_volatility * 1))

hist['higher_bound'] = hist.close * (1 + (hist.multiplier_volatility * 1))
hist['lower_bound'] = hist.close * (1 - (hist.multiplier_volatility * 1))

# hist['higher_bound'] = hist.close * (1 + (hist.multiplier_volatility * 0.6))
# hist['lower_bound'] = hist.close * (1 - (hist.multiplier_volatility * 0.6))

hist = labelling.get_barrier(hist,hist.close.values,hist.higher_bound.values,hist.lower_bound.values,window=50,side=None)

/var/folders/fx/9gkklqr57dgg7dv2kk03gpkw0000gn/T/ipykernel_30828/3375629895.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hist['multiplier'] = hist['exp_ema_pct_abs_shift_50']
/var/folders/fx/9gkklqr57dgg7dv2kk03gpkw0000gn/T/ipykernel_30828/3375629895.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hist['multiplier_volatility'] = hist['exp_pct_volatility_50']
/var/folders/fx/9gkklqr57dgg7dv2kk03gpkw0000gn/T/ipykernel_30828/3375629895.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resu

In [10]:
from stonks import labelling


ModuleNotFoundError: No module named 'stonks'

In [ ]:
window = 26

hist['multiplier'] = hist['exp_ema_pct_abs_shift_26']
hist['multiplier_volatility'] = hist['exp_pct_volatility_26']
hist['multiplier'] = hist['exp_ema_pct_abs_shift_50']
hist['multiplier_volatility'] = hist['exp_pct_volatility_50']

hist['higher_bound'] = hist.close * (1 + (hist.multiplier_volatility * 1))
hist['lower_bound'] = hist.close * (1 - (hist.multiplier_volatility * 1))

hist = labelling.get_barrier(hist,hist.close.values,hist.higher_bound.values,hist.lower_bound.values,window=26,side=None)

In [ ]:
(10/10 - 1) * 100

In [ ]:
(hist.multiplier_volatility * 0.7).describe()

In [11]:
hist['barrier'].value_counts()

 0.0    52892
 1.0    39318
-1.0    38666
Name: barrier, dtype: int64

In [12]:
hist['barrier'].loc[hist['barrier'] == 1] = 2
hist['barrier'].loc[hist['barrier'] == 0] = 1
hist['barrier'].loc[hist['barrier'] == -1] = 0

/var/folders/fx/9gkklqr57dgg7dv2kk03gpkw0000gn/T/ipykernel_30828/2354326595.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hist['barrier'].loc[hist['barrier'] == 1] = 2
/var/folders/fx/9gkklqr57dgg7dv2kk03gpkw0000gn/T/ipykernel_30828/2354326595.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hist['barrier'].loc[hist['barrier'] == 0] = 1
/var/folders/fx/9gkklqr57dgg7dv2kk03gpkw0000gn/T/ipykernel_30828/2354326595.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [ ]:
hist['barrier'].value_counts()

In [ ]:
# remove last N minutes of each day
hist = hist.loc[hist.minutes_open < (hist.minutes_open.max() - window)]

In [ ]:
# the prediction is equal to volatility shifted by the trend
# assume that I am trading on 100m timescale
# I want to predict, how much will the moving average move up in the next 100m?
# this can be done with a kind of regression
# alternatively, I can see how much the mean shifted from the previous 100m
# 

In [ ]:
# hist['barrier_1'] = hist.barrier
# hist['higher_bound'] = hist.close + hist.vol_std_100 * 3
# hist['lower_bound'] = hist.close - hist.vol_std_100 * 3
# hist = get_barrier(hist,hist.close.values,hist.higher_bound.values,hist.lower_bound.values,window=100)
# hist = hist.loc[hist.barrier_1 == 1]
# hist.barrier.loc[hist.barrier == -1] = 0

In [ ]:
# here we can train an ml model using the labels. lets leave this for now

In [ ]:
# lets generate an entry signal using existing features

# #hist['strategy1_buy'] = np.where(hist.MACD > 1, 1, 0)
# hist['strategy1_sell'] = 0
# # hist['strategy1_buy'] = np.where(hist.barrier == 1, 1, 0)
# # hist['strategy1_buy'] = np.where(hist.oversold_40 == 1, 1, 0)
# hist['strategy1_buy'] = 0
# # hist['strategy1_buy'].loc[(hist.MACD > 0) & (hist.macd_40_60 > 0) & (hist.Close_EMA_10) & (hist.EMA_10_EMA_30) & (hist.candle > 0) & (hist.macd > 0)] = 1
# # hist['strategy1_buy'].loc[(hist.oversold_14_30 == 1) & (hist.candle > 0) * (hist.MACD > 0)] = 1
# hist['strategy1_buy'].loc[(hist.MACD_Signal_MACD_5_8 == 1) & (hist.Close_EMA_100 == 1)] = 1
# hist['strategy1_sell'].loc[(hist.MACD_Signal_MACD_5_8 == 0)] = 1

#hist['strategy1_sell'].loc[(hist.overbought_14_70 == 1)] = 1


In [ ]:
#hist = hist.iloc[-70000:,:]

In [52]:
hist = hist.iloc[len(hist)//2:]

In [53]:
# from stonks.feature_selection import *

# # adjust step and scaling
feature_ranking = select_best_features(hist.drop(columns=['date','datetime','market_open','datetime_market_open']), 
                                       method='RFE', 
                                       estimator_type = 'random_forest', 
                                       #class_weight = {0:5,1:1,2:10},
                                       max_depth = 4, 
                                       n_features_to_select=20,
                                       step = 20)

# feature_ranking = select_best_features(hist.drop(columns=['date','datetime','market_open','datetime_market_open']), 
#                                        method='RFECV', 
#                                        estimator_type = 'random_forest', 
#                                        #class_weight = {0:5,1:1,2:10},
#                                        max_depth = 4, 
#                                        step = 20)


# feature_ranking = select_best_features(hist.drop(columns=['date','datetime','market_open','datetime_market_open']), 
#                                        method='genetic', 
#                                        estimator_type = 'random_forest', 
#                                        #class_weight = {0:5,1:1,2:10}, 
#                                        max_depth = 4)

#feature_ranking = select_best_features(hist,method='mutual_information')
#feature_ranking = select_best_features(hist.drop(columns=['date','datetime','market_open','datetime_market_open']), method='mutual_information')

ValueError: Found array with 0 sample(s) (shape=(0, 1186)) while a minimum of 1 is required by RFE.

In [ ]:
feature_ranking.sort_values().head(50).index

In [ ]:
f = pd.Series(feature_ranking,index=hist.drop(columns=['barrier','date','datetime','market_open','datetime_market_open']).columns)

In [ ]:
f.loc[f==1].index

In [ ]:
# RFE for F ['minutes_open', '30_min_bin', 'ATR_5', 'ATR_8', 'returns_vol_std_8',
    #    'ATR_12', 'returns_vol_std_12', 'ATR_20', 'returns_vol_std_20',
    #    'ATR_26', 'returns_vol_std_26', 'returns_vol_std_40',
    #    'exp_ema_pct_dir_shift_1500', 'ATR_5000', 'ATR_7000',
    #    'volume_macd_60_1500', 'ema_volume_vol_std_60_1500',
    #    'ema_volume_vol_std_new_60_1500', 'volume_macd_300_800',
    #    'ema_volume_vol_std_700_1500']

In [ ]:
# RFE
# ['minutes_open', '30_min_bin', 'last_30_min', 'returns_vol_std_8',
#        'returns_vol_std_12', 'returns_vol_std_20', 'returns_vol_std_26',
#        'returns_vol_std_40', 'exp_pct_volatility_150',
#        'exp_pct_volatility_200', 'exp_pct_volatility_600',
#        'volume_macd_50_100', 'ema_volume_vol_std_50_100',
#        'ema_volume_vol_std_new_50_100', 'volume_macd_50_200',
#        'ema_volume_vol_std_50_200', 'ema_volume_vol_std_new_50_200',
#        'volume_macd_60_150', 'ema_volume_vol_std_60_150',
#        'ema_volume_vol_std_new_60_150']

#RFECV
# ['minutes_open', 'ema_volume_vol_std_new_50_100', 'volume_macd_50_200',
#        'ema_volume_vol_std_60_150', 'ema_volume_vol_std_new_60_150',
#        'returns_vol_std_12', 'volume_macd_60_150', 'returns_vol_std_20',
#        'volume_macd_50_100', 'last_30_min', 'exp_pct_volatility_150',
#        '30_min_bin', 'exp_ema_pct_dir_shift_7000', 'exp_pct_volatility_600',
#        'returns_vol_std_40', 'volume_macd_60_1500', 'exp_pct_volatility_200',
#        'ema_volume_vol_std_50_100', 'ema_volume_vol_std_new_50_200',
#        'returns_vol_std_8', 'returns_vol_std_26', 'exp_pct_volatility_old_20',
#        'vol_std_volume_2', 'volume_macd_26_80', 'vol_std_volume_3',
#        'vol_std_volume_26', 'exp_ema_pct_abs_shift_1000', 'volume_macd_26_100',
#        'exp_pct_volatility_old_26', 'volume_macd_26_500',
#        'ema_volume_vol_std_26_500', 'trade_count', 'exp_ema_pct_abs_shift_100',
#        'returns_vol_std_50', 'returns_vol_std_5',
#        'ema_volume_vol_std_new_26_500', 'ema_volume_vol_std_50_200',
#        'vol_std_pct_26', 'volume_macd_20_150', 'vol_std_pct_12',
#        'exp_pct_volatility_100', 'vol_std_volume_20',
#        'ema_volume_vol_std_new_20_150', 'ema_volume_vol_std_new_60_1500',
#        'volume_macd_700_1500', 'exp_ema_pct_abs_shift_200',
#        'exp_ema_pct_abs_shift_150', 'ema_volume_vol_std_20_150',
#        'exp_pct_volatility_1000', 'exp_ema_pct_abs_shift_1200']

In [ ]:
pd.set_option('display.max_rows',1000)

In [ ]:
#model_features = ['vol_std_100','macd_40_60','RSI','returns_EMA_20','Close_EMA_50','Close_EMA_100','Close_EMA_10','MACD','RSI_100','barrier']
#model_features = ['RSI','vol_std_100','MACD','macd_40_60','macd3','barrier']
#model_features = ['MACD_Signal_MACD_5_8','Close_EMA_100','barrier']
# #model_features = ['MACD','RSI','vol_std_100','candle','barrier']

# # algo generated"
# model_features = ['lower_bound', 'returns_EMA_1200', 'ema5', 'higher_bound', 'RSI_100',
#        'swing_high_50', 'vol_std_50', 'vol_std_100', 'ATR_1000',
#        'macd_60_1440','barrier']

# # model_features = ['vol_std_100', 'vol_std_50', 'ATR_1000', 'ATR_200', 'ema5',
# #        'returns_EMA_200', 'macd_60_1440', 'macd_20_200', 'higher_bound',
# #        'close']

# # model_features = ['ema1', 'higher_bound', 'vol_std_50', 'ATR_1000', 'macd_60_1440']

# model_features = ['ATR_50', 'Close_EMA_30', 'Close_EMA_10', 'ema5', 'ema4', 'RSI_100',
#        'ATR_14', 'vol_std_100', 'ATR_1000', 'macd_60_1440','barrier']

# model_features = ['lower_bound', 'higher_bound', 'RSI_100', 'vol_std_100', 'ATR_1000',
#        'ATR_200', 'ATR_50', 'ema4', 'macd_60_1440', 'ema5','barrier']

# model_features = ['lower_bound', 'macd_300_500', 'macd_700_1500', 'macd_1500_15000',
#        'macd_7000_20000', 'ema5', 'macd_10000_45000', 'macd_45000_200000',
#        'ATR_1000', 'vol_std_100','barrier']

# genetic model found this
# model_features = ['macd_5_8', 'macd_8_26', 'macd_20_60_signal', 'macd_1500_15000_signal',
#         'macd_60_1440', 'ATR_14', 'ATR_50', 'vol_std_100', 'vol_std_50','barrier']

# model_features = ['macd_5_8', 'macd_8_26', 'macd_20_60_signal', 'macd_1500_15000_signal',
#         'macd_60_1440', 'ATR_14', 'ATR_50', 'vol_std_100', 'vol_std_50','vol_std_1000',
#         'volume_macd_20_150','volume_macd_5_8','volume_macd_700_1500',
#         'RSI_100','RSI_200','RSI_300','RSI',
#         'barrier']

# new weird features handpicked from the mutual information method
# model_features = ['swing_low_50', 'swing_high_50', 'swing_low_100', 'swing_high_100','swing_low_26', 'swing_high_26','swing_high_8', 'swing_low_8',
# 'ema_60000','ema_100000', 'ema_45000','ema5','ema_1000','ATR_200000','vol_std_200000','macd_10000_45000','barrier']

# new features from RFE step 40 with class weights
# model_features = ['volume_macd_50_200', 'volume_macd_60_1500', 'volume_macd_300_500',
#        'volume_macd_300_800', 'volume_macd_700_1500',
#        'volume_macd_1500_15000_signal', 'volume_macd_7000_20000_signal',
#        'ATR_8', 'ATR_12', 'ATR_20', 'ATR_26', 'ATR_40', 'RSI_200', 'RSI_250',
#        'vol_std_400', 'vol_std_500', 'vol_std_600', 'vol_std_700',
#        'vol_std_800', 'vol_std_1000', 'vol_std_1200', 'vol_std_1500',
#        'vol_std_1800', 'vol_std_2000', 'vol_std_2500', 'RSI_3000',
#        'vol_std_3000', 'vol_std_7000', 'vol_std_200000', 'ATR_14','barrier']

# # same but for the vol_std_100 labels
# model_features = ['macd_50_200', 'macd_700_1500', 'volume_macd_700_1500',
#        'macd_45000_100000', 'ema_20', 'vol_std_80', 'RSI_100', 'vol_std_100',
#        'RSI_150', 'vol_std_150', 'RSI_200', 'vol_std_200', 'RSI_250',
#        'RSI_300', 'ema_400', 'ATR_2500', 'vol_std_3000', 'ATR_5000',
#        'ema_9000', 'ema_10000', 'ema_12000', 'ATR_20000', 'vol_std_45000',
#        'RSI_80000', 'RSI_100000', 'ATR_150000', 'ATR_200000', 'RSI_200000',
#        'swing_low_200000', 'macd_20_200','barrier']

# model_features = ['volume_macd_700_1500', 'vol_std_80', 'vol_std_100', 'RSI_150',
#        'vol_std_150', 'RSI_200', 'ema_10000', 'vol_std_45000', 'RSI_80000',
#        'ATR_200000','barrier']

# lets start customizing the features by hand using maths and logic!

# model_features = ['ema_vol_std_100','vol_std_shift_100','vol_std_pct100','vol_std_pct_ratio100','barrier',
#                   'ema_vol_std_50','vol_std_shift_50','vol_std_pct50','vol_std_pct_ratio50',
#                 'ema_vol_std_200','vol_std_shift_200','vol_std_pct200','vol_std_pct_ratio200',
#                 'ema_vol_std_1000','vol_std_shift_1000','vol_std_pct1000','vol_std_pct_ratio1000',
#                 'macd_50_200', 'macd_700_1500','macd_20_200', 'multiplier']

# hist.macd
# [12,26],[12,50],[12,100]

# this bad boy gave me 19% month 800% Year. 28% with volatility multiplier take profit
model_features = ['ema_vol_std_50','vol_std_shift_50','vol_std_pct50','vol_std_pct_ratio50',
'vol_std_shift_12_100','vol_std_shift_new12_100','vol_std_pct_ratio_12_100',
'vol_std_shift_12_26','vol_std_shift_new12_26','vol_std_pct_ratio_12_26','vol_std_shift_12',
'min_vol_std12', 'max_vol_std12', 'ema_vol_std_12',
'macd_12_26','macd_12_50','macd_12_100',
'barrier','multiplier']

# updated names
model_features = ['close_ema_vol_std_50',
 'ema_vol_std_shift_50',
 'vol_std_pct_50',
 'vol_std_pct_ratio_50',
 'ema_vol_std_shift_12_100',
 'ema_vol_std_shift_new_12_100',
 'vol_std_pct_ratio_12_100',
 'ema_vol_std_shift_12_26',
 'ema_vol_std_shift_new_12_26',
 'vol_std_pct_ratio_12_26',
 'ema_vol_std_shift_12',
 'min_vol_std_12',
 'max_vol_std_12',
 'close_ema_vol_std_12',
 'macd_12_26',
 'macd_12_50',
 'macd_12_100',
 'barrier',
 'exp_ema_pct_shift_12']

# model_features = ['close_ema_vol_std_50',
#  'ema_vol_std_shift_50',
#  'vol_std_pct_50',
#  'vol_std_pct_ratio_50',
#  'ema_vol_std_shift_12_100',
#  'ema_vol_std_shift_new_12_100',
#  'vol_std_pct_ratio_12_100',
#  'ema_vol_std_shift_12_26',
#  'ema_vol_std_shift_new_12_26',
#  'ema_vol_std_shift_new_26_500',
#  'vol_std_pct_ratio_12_26',
#  'ema_vol_std_shift_12',
#  'min_vol_std_12',
#  'max_vol_std_12',
#  'close_ema_vol_std_12',
#  'close_ema_vol_std_26',
#  'macd_12_26',
#  'macd_12_50',
#  'macd_12_100',
#  'macd_26_100',
#  'barrier',
# #  'exp_ema_pct_abs_shift_12',
# #  'exp_ema_pct_dir_shift_12',
#  'exp_ema_pct_abs_shift_50',
#  'exp_ema_pct_dir_shift_50',
#  'RSI_12',
#  'RSI_26',
#  'volume_macd_60_1500',
#  'volume_macd_26_500',
#  'volume_macd_20_150',
#  'ATR_12',
#  'ATR_5',
#  'ATR_26',
#  'exp_pct_volatility_400',
#  #'swing_high_50','swing_low_50',
#  'min_vol_std_50','max_vol_std_50',
#  'exp_ema_pct_abs_shift_800','exp_ema_pct_dir_shift_5000',
#  'returns_vol_std_5', 'returns_vol_std_8', 'returns_vol_std_12',
#        'exp_pct_volatility_old_12', 'returns_vol_std_20', 'returns_vol_std_26',
#        'returns_vol_std_40', 'exp_ema_pct_abs_shift_200', 'volume_macd_50_100',
#        'volume_macd_60_150'
#  ]

# PLTR
model_features = ['close_ema_vol_std_50',
        'ema_vol_std_shift_50',
        'vol_std_pct_50',
        'vol_std_pct_ratio_50',
        'ema_vol_std_shift_12_100',
        'ema_vol_std_shift_new_12_100',
        'vol_std_pct_ratio_12_100',
        'ema_vol_std_shift_12_26',
        'ema_vol_std_shift_new_12_26',
        'ema_vol_std_shift_new_26_500',
        'vol_std_pct_ratio_12_26',
        'ema_vol_std_shift_12',
        'min_vol_std_12',
        'max_vol_std_12',
        'close_ema_vol_std_12',
        'close_ema_vol_std_26',
        'macd_12_26',
        'macd_12_50',
        'macd_12_100',
        'macd_26_100',
        'exp_ema_pct_abs_shift_50',
        'exp_ema_pct_dir_shift_50',
        'RSI_12',
        'RSI_26',
        'volume_macd_60_1500',
        'volume_macd_26_500',
        'volume_macd_20_150',
        'ATR_12',
        'ATR_5',
        'ATR_26',
        'exp_pct_volatility_50',
        'barrier'
        ]
# trying to fix overfitting
model_features = ['close_ema_vol_std_50',
        'ema_vol_std_shift_50',
        'vol_std_pct_50',
        'vol_std_pct_ratio_50',
        'ema_vol_std_shift_12_100',
        'ema_vol_std_shift_new_12_100',
        'vol_std_pct_ratio_12_100',
        'ema_vol_std_shift_12_26',
        'ema_vol_std_shift_new_12_26',
        'ema_vol_std_shift_new_26_500',
        'vol_std_pct_ratio_12_26',
        'ema_vol_std_shift_12',
        'min_vol_std_12',
        'max_vol_std_12',
        'close_ema_vol_std_12',
        'close_ema_vol_std_26',
        'macd_12_26',
        'macd_12_50',
        'macd_12_100',
        'macd_26_100',
        'exp_ema_pct_abs_shift_50',
        'exp_ema_pct_dir_shift_50',
        'RSI_12',
        'RSI_26',
        'volume_macd_60_1500',
        'volume_macd_26_500',
        'volume_macd_20_150',
        'exp_pct_volatility_50',
        '30_min_bin',
        'pct_change_daily',
        # 'first_30_min',
        # 'last_30_min',
        'barrier'
        ]

# PLTR
model_features = [
        #'minutes_open', 
        #'ema_volume_vol_std_new_50_100',
        #'returns_vol_std_12', 'returns_vol_std_26','returns_vol_std_40',
        'close_ema_vol_std_26',
        'close_ema_vol_std_50',
        'min_vol_std_12',
        'max_vol_std_12',
        # 'min_vol_std_50',
        # 'max_vol_std_50',
        'RSI_26',
        'RSI_50',
        'ema_volume_vol_std_60_1500',
        'ema_volume_vol_std_26_500',
        'ema_volume_vol_std_20_150',
        '30_min_bin',
        'pct_change_daily',
        'barrier'
]

# F to pay respects
model_features = ['minutes_open', '30_min_bin', 'ATR_3', 'ATR_5', 'ATR_8',
       'returns_vol_std_8', 'ATR_12', 'returns_vol_std_12', 'ATR_20',
       'returns_vol_std_20', 'ATR_26', 'returns_vol_std_26', 'ATR_40',
       'returns_vol_std_40', 'returns_vol_std_50', 'ATR_5000',
       'volume_macd_60_1500', 'ema_volume_vol_std_60_1500',
       'ema_volume_vol_std_new_60_1500', 'volume_macd_300_800','barrier']

# # minimal manual
# model_features = ['volume_macd_60_150', 'volume_macd_60_1500', 'returns_vol_std_40', 'exp_ema_pct_abs_shift_50', 'exp_ema_pct_abs_shift_200', 'exp_ema_pct_dir_shift_50',
#                    'min_vol_std_50','max_vol_std_50', 'ema_vol_std_shift_new_12_100', 'ema_vol_std_shift_new_26_500','barrier']

# feature selection - mutual information
#model_features = ['exp_pct_volatility_400','swing_high_50','swing_low_50','exp_ema_pct_abs_shift_800','exp_ema_pct_dir_shift_5000','barrier']

# with half set and smallet take profit - 
# model_features = ['higher_bound','lower_bound','multiplier_volatility','multiplier','returns_EMA_50','returns_EMA_200','returns_EMA_1200','ema_vol_std_shift_new_300_800','vol_std_pct_ratio_300_800','volume_macd_300_800','macd_60_150','barrier']
# model_features = ['returns_vol_std_12', 'returns_vol_std_20', 'returns_vol_std_26',
#        'exp_pct_volatility_500', 'exp_pct_volatility_600',
#        'exp_pct_volatility_1000', 'exp_pct_volatility_old_1200',
#        'exp_pct_volatility_1500', 'vol_std_pct_ratio_7000',
#        'min_vol_std_7000','barrier']
# feature selection - RFE
# model_features = ['returns_vol_std_5', 'returns_vol_std_8', 'returns_vol_std_12',
#        'exp_pct_volatility_old_12', 'returns_vol_std_20', 'returns_vol_std_26',
#        'returns_vol_std_40', 'exp_ema_pct_abs_shift_200', 'volume_macd_50_100',
#        'volume_macd_60_150','barrier']

#model_features = ['macd_12_26', 'macd_26_100', 'close_ema_vol_std_26','barrier', 'RSI_12','RSI_26']

# manual random forest optimisation gave me this with 11%
# model_features = ['vol_std_shift_new60_1500',
#  'RSI_1500',
#  'vol_std_pct_ratio_50_200',
#  'ema_vol_std_80',
#  'RSI_14',
#  'ema_vol_std_12',
#  'multiplier',
#  'RSI_8',
#  'ema_vol_std_20',
#  'RSI_12',
#  'barrier']

 # hybrid
# 15%
# model_features = ['ema_vol_std_50','vol_std_shift_50','vol_std_pct50','vol_std_pct_ratio50',
# 'vol_std_shift_12_100','vol_std_shift_new12_100','vol_std_pct_ratio_12_100',
# 'vol_std_shift_12_26','vol_std_shift_new12_26','vol_std_pct_ratio_12_26','vol_std_shift_12',
# 'min_vol_std12', 'max_vol_std12', 'ema_vol_std_12',
# 'macd_12_26','macd_12_50','macd_12_100',
# 'RSI_12', 'ema_vol_std_20', 'vol_std_pct_ratio_50_200','ema_vol_std_80','vol_std_shift_new60_1500',
# 'barrier','multiplier']

# 21% crazy stuff
# model_features = ['multiplier','ema_vol_std_12','RSI_12','vol_std_shift_new12_100',
# 'vol_std_pct_ratio_12_100','vol_std_pct50','ema_vol_std_50','barrier']

# # try for the 50 min 
# model_features = ['multiplier','ema_vol_std_12','RSI_12','vol_std_shift_new12_100',
# 'vol_std_pct_ratio_12_100','vol_std_pct50','ema_vol_std_50',
# 'ema_vol_std_26', 'ema_vol_std_100', 'vol_std_pct100','vol_std_shift_new50_200',
# 'vol_std_pct_ratio_50_200','RSI_50', 'barrier']


# model_features = ['ema_vol_std_1000','vol_std_shift_1000','vol_std_pct1000','vol_std_pct_ratio1000','barrier',
#                   'ema_vol_std_500','vol_std_shift_500','vol_std_pct500','vol_std_pct_ratio500',
#                 'ema_vol_std_2000','vol_std_shift_2000','vol_std_pct2000','vol_std_pct_ratio2000',
#                 'ema_vol_std_10000','vol_std_shift_10000','vol_std_pct10000','vol_std_pct_ratio10000',
#                 'macd_5000_2000', 'macd_7000_15000','macd_200_2000']
#model_features = ['volume_macd_60_1500','volume_macd_26_500','volume_macd_20_150','ATR_12','ATR_5','ATR_26','barrier']

#model_features = hist.columns

In [ ]:
model_features = ['minutes_open', '30_min_bin', 'ATR_5', 'ATR_8', 'returns_vol_std_8',
       'ATR_12', 'returns_vol_std_12', 'ATR_20', 'returns_vol_std_20',
       'ATR_26', 'returns_vol_std_26', 'returns_vol_std_40',
       'exp_ema_pct_dir_shift_1500', 'ATR_5000', 'ATR_7000',
       'volume_macd_60_1500', 'ema_volume_vol_std_60_1500',
       'ema_volume_vol_std_new_60_1500', 'volume_macd_300_800',
       'ema_volume_vol_std_700_1500','barrier']

In [18]:
# new pct based tests 
# vol_std_pct_12 - describes general volatility
# swing_low_pct_ - 
# swing_high_pct_ - 
# close_ema_pct_ - 
# volume_macd_pct - 
# macd_pct - 
# candle pct - 
# vol_std_pct_ratio_ - describes the shift in volatility
# hist['multiplier'] = hist['exp_ema_pct_abs_shift_50']
# hist['multiplier_volatility'] = hist['exp_pct_volatility_50']

model_features = ['minutes_open', '30_min_bin',
                  'vol_std_pct_12', 'vol_std_pct_26', 'vol_std_pct_50', 'vol_std_pct_100',
                  'swing_low_pct_12', 'swing_low_pct_26', 'swing_low_pct_50', 'swing_low_pct_100', 
                  'swing_high_pct_12', 'swing_high_pct_26', 'swing_high_pct_50', 'swing_high_pct_100',
                  'close_ema_pct_12', 'close_ema_pct_26', 'close_ema_pct_50', 'close_ema_pct_100',
                  'volume_macd_pct_12_26', 'volume_macd_pct_12_50', 'volume_macd_pct_12_100', 'volume_macd_pct_26_100', 
                  'macd_pct_12_26', 'macd_pct_12_50', 'macd_pct_12_100', 'macd_pct_26_100',
                  'candle_pct',
                  'exp_ema_pct_abs_shift_50',
                  'exp_pct_volatility_50',
                  'barrier']

In [41]:
model_features = ['swing_high_pct_12', 'volume_macd_pct_12_100',
       'swing_low_pct_50', 'macd_pct_26_100',
       'swing_low_pct_100', 'swing_high_pct_100', 'vol_std_pct_100',
       'exp_ema_pct_abs_shift_50', 'swing_high_pct_26',
       'exp_pct_volatility_50', 'volume_macd_pct_26_100', '30_min_bin',
       'vol_std_pct_26', 'minutes_open','barrier']

In [ ]:
#hist_sample = hist.iloc[-60000:]

In [14]:
import importlib
importlib.reload(models)

<module 'models' from '/Users/gluzman/Desktop/Code/stonks/stonks/models.py'>

In [ ]:
importlib.reload(models)

In [42]:
import models
# 0.48 and 0.46 precision record!
# try the min max scaler
# try boosting with all features
# try boosting with no depth - boosting with no depth and all features achieves 0.40 0.78 P R!

# without parallel = 224m
# the XGboost parallelisation cut it down to 17m
# with new 500-600 features and max depth 3 it took me 50 min....
#ypred, model_index = predict_barrier(hist, features = model_features, max_depth=None, model_type='boosting', parallel=False)

# try - subsample = 0.8, learning rate up to 0.3, reg lambda, reg alpha to small positive value (regularisation)
# ypred, model_index,model = models.predict_barrier(hist, features = model_features, max_depth=None, model_type='boosting', parallel=True, tree_method = 'hist')

# this baby gave me 0.38 0.97!
#ypred, model_index,model = models.predict_barrier(hist, features = model_features, max_depth=6, model_type='random_forest', parallel=True, class_weight={0:5,1:1,2:10})
# ypred, model_index,model = models.predict_barrier(hist, 
#                                                   features = model_features, 
#                                                   max_depth=4, 
#                                                   model_type='random_forest', 
#                                                   parallel=True, 
#                                                   #class_weight={0:1,1:10},
#                                                   class_weight='balanced',
#                                                   normalise=False,
#                                                   prob_thresh = None,
#                                                   split_size = 0.9)

ypred, model_index,model = models.predict_barrier(hist, 
                                                  features = model_features, 
                                                  max_depth=4, 
                                                  model_type='boosting', 
                                                  parallel=True, 
                                                  normalise=False,
                                                  prob_thresh = None,
                                                  tree_method='hist',
                                                  split_size=0.90,
                                                  sample=False,
                                                  sampling_rate=8,
                                                  prod_model=False)

              precision    recall  f1-score   support

         0.0       0.42      0.37      0.39      2612
         1.0       0.64      0.84      0.73      3805
         2.0       0.41      0.27      0.33      2527

    accuracy                           0.54      8944
   macro avg       0.49      0.49      0.48      8944
weighted avg       0.51      0.54      0.52      8944



In [ ]:
# ideas - try % change volatility
# try set barrier manually as in 1%
# try one sided barrier - buy only
# try pick timescale where volatility > 1%
# add pct_change volatility vol_std/close price
# how many volatility units above or below the EMA are we? 
# eg strategy - if we are 0.5 volatility units below the appropriate ema buy and hold until we hit 1 volatility units
# we can also add future volatility forecasting

# my barrier method may be interfering with forecasting. as it is random whether it will hit the -1 or + 1 std
# a simple algo should work like this - if we are 1 volatility below average then we know it is statistically likely to rebound
# this variable should be enough to predict if we will hit the buy barrier
# we can add trend confirmation to rule out weird cases where we are likely to go further down


# try - will it go up 1% in the next 1000m?
# use probability distribution to model volatility!
# mean and std describe the dsitribution
# we can combine it with trend to describe the time series
# we can use the information about current stock price as a prior + normal distribution to get the probability that we will hit a certain % return within a certain time window
# also find out about stochastic volatility and GARCH

#One way to incorporate the current stock price into your model is by using a method called Bayesian updating. 
#This involves using the prior distribution (the distribution you estimated from historical data) and updating it with new data (the current stock price) 
# to get a posterior distribution that takes into account the current information.
#Specifically, you can use the current stock price to calculate the likelihood of observing that price given the parameters of your prior distribution. 
# This likelihood can then be used to update the prior distribution to get the posterior distribution. 
# The posterior distribution will then take into account both the historical information and the current stock price.
# Once you have the posterior distribution, you can use it to calculate various metrics, such as the expected return or 
# the probability of the stock price crossing a certain threshold.

# experiment with combining volatility information over different timescales - 100m, 1000m to deconcstruct the time series
# explore what is stochastic volatility

# express all information in terms of deviation from ema (% change or std deviations)
# the close > ema indicator is best as an stds above mean indicator

# in order to describe not-normal distributions and identify support levels we can calculate number of points that crossed the N stds barrier (% outliers)
# or the box and whisker stuff




In [43]:
pd.set_option('display.max_rows',1000)
pd.Series(model.feature_importances_,index=model.feature_names_in_).sort_values().tail(100)

volume_macd_pct_12_100      0.032426
swing_high_pct_12           0.033397
macd_pct_26_100             0.040128
swing_low_pct_50            0.043929
30_min_bin                  0.047208
swing_low_pct_100           0.048772
swing_high_pct_100          0.049254
vol_std_pct_100             0.050261
exp_ema_pct_abs_shift_50    0.059934
exp_pct_volatility_50       0.068285
swing_high_pct_26           0.068920
volume_macd_pct_26_100      0.077306
vol_std_pct_26              0.096571
minutes_open                0.283608
dtype: float32

In [ ]:
pd.Series(model.feature_importances_,index = model.feature_names_in_).sort_values(ascending=False).head(50)

In [ ]:
pd.Series(model.feature_importances_,index = model.feature_names_in_).sort_values(ascending=False).head(50)

In [ ]:
# ypred_experiment = pd.DataFrame(ypred,columns = ['0','1','2'])
# ypred_experiment['buy'] = 0
# ypred_experiment['buy'].loc[ypred_experiment['2']>0.50] = 1
# ypred_experiment['buy_proba'] = 0
# ypred_experiment['buy'].loc[ypred_experiment['2']>0.60] = 1
# ypred_experiment.sum()


In [44]:
backtest_hist = hist.loc[model_index]
backtest_hist['ml_prediction'] = ypred

/var/folders/fx/9gkklqr57dgg7dv2kk03gpkw0000gn/T/ipykernel_30828/3322428535.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  backtest_hist['ml_prediction'] = ypred


In [45]:
backtest_hist.ml_prediction.value_counts()

1    4965
0    2298
2    1681
Name: ml_prediction, dtype: int64

In [46]:
backtest_hist['strategy1_sell'] = 0
backtest_hist['strategy1_buy'] = 0
#backtest_hist['strategy1_buy'].loc[(backtest_hist.ml_prediction == 1)] = 1
backtest_hist['strategy1_buy'].loc[(backtest_hist.ml_prediction == 2)] = 1
#backtest_hist['strategy1_buy'].loc[(backtest_hist.ml_prediction == 1)] = 1
#backtest_hist['strategy1_buy'].loc[(backtest_hist.ema_vol_std_100 < -0.3)] = 1
# #hist['strategy1_buy'].loc[(hist.ml_prediction == 1) & (hist.ml_prediction2 == 1)] = 1
# #hist['strategy1_buy'].loc[(hist.ml_prediction == 1) & (hist.Close_EMA_100 == 1)] = 1
#backtest_hist['strategy1_sell'].loc[(backtest_hist.ml_prediction == 0)] = 1
# # hist['strategy1_buy'].loc[(hist.ml_prediction != -1)] = 1

#hist['strategy1_buy'].loc[(hist.barrier == 2)] = 1


/var/folders/fx/9gkklqr57dgg7dv2kk03gpkw0000gn/T/ipykernel_30828/3022783643.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  backtest_hist['strategy1_sell'] = 0
/var/folders/fx/9gkklqr57dgg7dv2kk03gpkw0000gn/T/ipykernel_30828/3022783643.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  backtest_hist['strategy1_buy'] = 0
/var/folders/fx/9gkklqr57dgg7dv2kk03gpkw0000gn/T/ipykernel_30828/3022783643.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentat

In [ ]:
backtest_hist['strategy1_sell'].loc[(backtest_hist.ml_prediction == 0)] = 1

In [47]:
backtest_hist.strategy1_buy.value_counts()


0    7263
1    1681
Name: strategy1_buy, dtype: int64

In [ ]:
# try to use the MACD signal line
# combine short term and long term trend indicators
# eg default MACD (13 26?) and 100 EMA
# if price is above 100 EMA, the trend is up
# if price is below 100 EMA, the trend is down
# wait for MACD crossover and enter
# set stop loss at recent swing-low and take profit at 1.5

# crossovers only work on trending markets. what metric is there of a trending market?
# use support and resistance levels or RSI?

# problem - EMA window and MACD window may vary by month, day, time of market etc.

In [ ]:
# record
# record - 64% return! interesting that a 1/3 stop loss gives low winrate of 0.29 but high win size
# the oversold + candle strat gives 70% return. interesting

# by tuning vol std I increased returns to 4% per trade at 40% winrate. gotta be smart with the stop loss/ take profit.
# need to add visualisation to monitor ovsious stuff like this

# got 93% return by using 5 min bars, swing low 50 * 1/ 1.5 barrier 100, macd signal + close ema 100

In [48]:
# backtest_hist['higher_bound'] = backtest_hist.close + backtest_hist.vol_std_100 * 1
# backtest_hist['lower_bound'] = backtest_hist.close - backtest_hist.vol_std_100 * 1

# hist['multiplier_volatility'] = hist['vol_std_' + str(window)]/hist['ema_' + str(window)].ewm(window * 10).mean()
# # try flat multiplier here
# # try flat % gain on closing price

backtest_hist['multiplier'] = backtest_hist['exp_ema_pct_abs_shift_50']
backtest_hist['multiplier_volatility'] = backtest_hist['exp_pct_volatility_50']
#backtest_hist['multiplier_volatility'] = backtest_hist['exp_pct_volatility_50'] * 0.8

backtest_hist['multiplier'] = backtest_hist['exp_ema_pct_abs_shift_26']
backtest_hist['multiplier_volatility'] = backtest_hist['exp_pct_volatility_26']
#backtest_hist['multiplier_volatility'] = backtest_hist['exp_pct_volatility_old_12']

# PLTR strat
backtest_hist['higher_bound'] = backtest_hist.close * (1 + ((0 * backtest_hist.multiplier + 2 * backtest_hist.multiplier_volatility) * 1))
backtest_hist['lower_bound'] = backtest_hist.close * (1 - ((0 * backtest_hist.multiplier + 0.4 * backtest_hist.multiplier_volatility) * 1))

# backtest_hist['higher_bound'] = backtest_hist.close * (1 + ((0 * backtest_hist.multiplier + 3 * backtest_hist.multiplier_volatility) * 1))
# backtest_hist['lower_bound'] = backtest_hist.close * (1 - ((0 * backtest_hist.multiplier + 0.4 * backtest_hist.multiplier_volatility) * 1))

# backtest_hist['higher_bound'] = backtest_hist.close * (1 + ((0 * backtest_hist.multiplier + 3 * backtest_hist.multiplier_volatility) * 1))
# backtest_hist['lower_bound'] = backtest_hist.close * (1 - ((0 * backtest_hist.multiplier + 1 * backtest_hist.multiplier_volatility) * 1))

In [49]:
# new record!
# bitcoin strat, 12 min forecast window, historical ema 12/ema 12 multiplier, 73% winrate
# 

In [50]:
import importlib
import backtest
importlib.reload(backtest)

<module 'backtest' from '/Users/gluzman/Desktop/Code/stonks/stonks/backtest.py'>

In [51]:
trades = backtest.run_backtest(
    backtest_hist=backtest_hist.loc[backtest_hist.lower_bound.notna()], 
    buy_index=backtest_hist.loc[backtest_hist.strategy1_buy==1].index, 
    #sell_index=backtest_hist.loc[backtest_hist.strategy1_sell==1].index,
    sell_index=[],
    take_profit_series=backtest_hist.higher_bound,
    stop_loss_series=backtest_hist.lower_bound,
    risk=1,
    cash = 10000,
    holding_period= 26,
    min_holding_period=0,
    hold_on_buy_signal=True,
    sltp=True,
    sltp_update=False)

Cash money:
8777.093127574664
99
99
Winrate:
0.30303030303030304
Avg win:
0.011095550029493634
Avg loss:
-0.007563242196508188


In [ ]:
{
    'symbol': 'PLTR',
    'feature_names': [],
    'labelling': {
        'method': 'triple_barrier',
        'forecast_window': 50,
        'multiplier_lower': {
            'window': 50,
            'feature_name_mean': 'exp_ema_abs_shift',
            'feature_name_vol': 'exp_vol',
            'mean_multiplier': 1,
            'vol_multiplier': 2
        },
        'multiplier_higher': {
            'window': 50,
            'feature_name_mean': 'exp_ema_abs_shift',
            'feature_name_vol': 'exp_vol',
            'mean_multiplier': 1,
            'vol_multiplier': 2
        },
    },
    'sltp': {
        'multiplier_lower': {
            'window': 50,
            'feature_name_mean': 'exp_ema_abs_shift',
            'feature_name_vol': 'exp_vol',
            'mean_multiplier': 1,
            'vol_multiplier': 2
        },
        'multiplier_higher': {
            'window': 50,
            'feature_name_mean': 'exp_ema_abs_shift',
            'feature_name_vol': 'exp_vol',
            'mean_multiplier': 1,
            'vol_multiplier': 2
        }
    },
    'backtest': {
        'starting_cash': 10000,
        'sell_index': False,
        'sltp': True,
        'sltp_update': False,
        'risk': 1,
        'holding_period': 26,
        'min_hold': 0,
        'hold_on_buy': False
    }   
}

In [ ]:
# import pickle
# filename = 'stonks/models/PLTR.pkl'
# pickle.dump(model, open(filename, 'wb'))

In [ ]:
# experiment with hold duration - max_hold param, separate rules for selling on take profit and stop loss

In [ ]:
10000 * (1.30**12)

In [ ]:
trades['date'] = trades.buy_index.apply(lambda x: x.date())

In [ ]:
trades.tail(50)

In [ ]:
summary = trades.groupby('date')['returns'].agg(['mean','max','min','count'])
summary['max_hold'] = trades.groupby('date')['holding_duration'].max()
summary['$ gain'] = trades.groupby('date')['$ gain'].sum()

In [ ]:
summary

In [ ]:
# two tasks

# raw strategy
# iterate over list of MACD, close_ema, RSI, ATR etc features
# create buy signals and sell signals from those
# run backtest for each
# record performance for total, month, week, day

# alternative
# search for optimal features separately - what tends to happen just before a barrier is hit? faster
# but this is pretty bad because it completely ignored feature combinations, they are essential
# solution - iterate over manual combinations or use trees

# which strategy is optimal for which barrier params?

# try ml - which ml feature x barrier x stop loss combination produces the best precision, recall, returns?

# examples - macd5, 8, 20, 200, X 5min X 15 min, x hour
# close_EMA
# overbought
# oversold
# candle
# signal 

In [ ]:
show_trades(hist,trades,day='2023-04-11')